## Librerias


In [3]:
!pip install fuzzywuzzy
import pandas as pd
from fuzzywuzzy import fuzz

#Google colab
from google.colab import auth
from google.cloud import bigquery

from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Mounted at /content/drive


## Base

In [5]:
#base 1
base1= pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base1.csv')
#base 2
base2= pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base2.csv')

### **Base 1**

- campaign: Nombre del evento enviada por web service
- customer: Nombre del comercio (eticket/tuboleta)
- creation_date: timestamp de la primera Alcancía creada

### **Base 2**
- campaign: Nombre del evento creada por Armatuvaca.
- campaign_slug: 	slug de las campañas.
- customer: Nombre del comercio (eticket/tuboleta)
- creation_date: timestamp de la fecha de creación del evento.


In [7]:
import pandas as pd
from fuzzywuzzy import fuzz

def match_eventos(base1, base2, umbral_similitud=80, dias_diferencia=15):
    matches = []

    for _, row1 in base1.iterrows():
        nombre1 = str(row1['campaign']).lower()
        fecha1 = pd.to_datetime(row1['creation_date'], unit='s')

        # Lista para almacenar posibles matches para este registro de base1
        matches_evento = []

        for _, row2 in base2.iterrows():
            fecha2 = pd.to_datetime(row2['creation_date'], unit='s')

            # Calcular diferencia en días
            diferencia_dias = abs((fecha2 - fecha1).days)

            # Calcular similitud de nombre
            similitud_nombre = fuzz.ratio(nombre1, str(row2['campaign']).lower())

            # Procesar el slug: remover guiones y convertir a minúsculas
            slug2 = str(row2['campaign_slug']).lower().replace('-', ' ')

            # Calcular similitud de slugs con el nombre usando ratio de Levenshtein
            similitud_slug = fuzz.ratio(nombre1, slug2)

            # Criterios de matching
            if (similitud_nombre > umbral_similitud or similitud_slug > 70) and \
               (diferencia_dias <= dias_diferencia):
                matches_evento.append({
                    'campaign_base1': row1['campaign'],
                    'customer_base1': row1['customer'],
                    'campaign_base2': row2['campaign'],
                    'customer_base2': row2['customer'],
                    'fecha_creacion_base1': fecha1,
                    'fecha_creacion_base2': fecha2,
                    'campaign_slug_base2': row2['campaign_slug'],
                    'similitud_nombre': similitud_nombre,
                    'similitud_slug': similitud_slug,
                    'diferencia_dias': diferencia_dias
                })

        # Si hay matches, seleccionar solo el mejor
        if matches_evento:
            # Ordenar por similitud de nombre y slug, de mayor a menor
            matches_evento_ordenados = sorted(matches_evento,
                                              key=lambda x: (x['similitud_nombre'], x['similitud_slug']),
                                              reverse=True)

            # Agregar solo el mejor match
            matches.append(matches_evento_ordenados[0])

    return pd.DataFrame(matches)

# Cargar las bases de datos
base1= pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base1.csv')
base2= pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base2.csv')

# Crear DataFrame de matches
df_matches = match_eventos(base1, base2)

# Ordenar resultados
df_matches = df_matches.sort_values(
    ['similitud_nombre', 'similitud_slug', 'diferencia_dias'],
    ascending=[False, False, True]
)

# Mostrar resumen de resultados
print("\nNúmero de matches:", len(df_matches))


Número de matches: 125


In [8]:
df_matches

,campaign_base1,customer_base1,campaign_base2,customer_base2,fecha_creacion_base1,fecha_creacion_base2,campaign_slug_base2,similitud_nombre,similitud_slug,diferencia_dias
16,ARTBAT,Tuboleta,Artbat,Tuboleta,2023-11-28 03:29:25,2023-11-29 18:54:28,artbat,100,100,1
21,ANDREA BOCELLI,eticket,Andrea Bocelli,eticket,2024-11-07 22:24:52,2024-11-07 13:39:41,andrea-bocelli,100,100,1
124,THE JUANPIS LIVE SHOW: MAKE COLOMBIA GREAT AGAIN,eticket,The Juanpis Live Show: Make Colombia Great Again,eticket,2024-07-31 23:13:13,2024-07-26 16:39:12,juanpis-live-show-make-colombia-great-again-bo...,100,83,6
25,THE BEATLES SYMPHONIC FANTASY,Tuboleta,The Beatles Symphonic Fantasy,Tuboleta,2024-08-09 01:41:37,2024-08-02 22:05:19,the-beatles-symphonic-fantasy-bogota-2024,100,83,7
69,THE JUANPIS LIVE SHOW: LOS NOTO ASUSTADOS TEMP...,eticket,The Juanpis Live Show: Los Noto Asustados Temp...,eticket,2024-04-02 13:09:02,2024-04-04 17:41:45,juanpis-live-show-los-noto-asustados-2-bogota-...,100,80,2
...,...,...,...,...,...,...,...,...,...,...
23,JUANES PEREIRA,eticket,Juanes,eticket,2023-12-02 02:41:51,2023-11-30 22:06:11,juanes-pereira-2024,60,85,2
95,BAUM FESTIVAL 2024 PROMOTORES,eticket,Baum Festival,NaN,2023-11-20 14:08:25,2023-11-28 00:35:54,baum-festival-2024,60,75,7
75,BRESH BOGOTÁ,eticket,Bresh,eticket,2024-03-26 17:00:05,2024-03-21 17:17:32,bresh-bogota-2024,59,76,5
114,JUANES BUCARAMANGA,eticket,Juanes,eticket,2023-11-22 18:55:49,2023-11-30 22:11:29,juanes-bucaramanga-2024,50,88,8
